In [ ]:
!pip install nltk scikit-learn wordcloud transformers torch streamlit pyngrok openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from wordcloud import WordCloud

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_excel("chatgpt_style_reviews_dataset.xlsx")
df.head()

,date,title,review,rating,username,helpful_votes,review_length,platform,language,location,version,verified_purchase
0,########,Review title 1,"Not satisfied, many bugs and issues.",1,user1,80,6,Amazon,zh,Kenya,2.1.4,No
1,########,Review title 2,Amazing quality and user-friendly interface.,5,user2,180,5,Flipkart,zh,France,1.2.3,No
2,########,Review title 3,"Terrible experience, needs major improvements.",2,user3,154,5,Flipkart,pt,USA,1.2.3,No
3,########,Review title 4,Poor performance and not user-friendly.,1,user4,96,5,Amazon,es,Qatar,2.1.4,Yes
4,########,Review title 5,"Not satisfied, many bugs and issues.",2,user5,139,6,Website,ar,Kenya,2.1.4,No


In [ ]:
#DATA PREPROCESSING (TEXT CLEANING)
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["clean_review"] = df["review"].apply(clean_text)

In [ ]:
#SENTIMENT LABEL CREATION
def rating_to_sentiment(r):
    if r >= 4:
        return "Positive"
    elif r == 3:
        return "Neutral"
    else:
        return "Negative"

df["sentiment"] = df["rating"].apply(rating_to_sentiment)
df["sentiment"].value_counts()

,count
sentiment,
Positive,200
Negative,195
Neutral,105


In [ ]:
#EDA

In [ ]:
#ML MODEL (TF-IDF + LOGISTIC REGRESSION)
X = df["clean_review"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

lr = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

lr.fit(X_train_tfidf, y_train)
y_pred = lr.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        39
     Neutral       1.00      1.00      1.00        21
    Positive       1.00      1.00      1.00        40

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [ ]:
#TRANSFORMER MODEL (BERT – COMPARISON MODEL)
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
#BERT Embeddings Function
def bert_embedding(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:,0,:].numpy()

In [ ]:
#Train/Test with BERT (NO BALANCING)
X_train_bert = np.vstack(X_train.apply(bert_embedding))
X_test_bert  = np.vstack(X_test.apply(bert_embedding))

bert_lr = LogisticRegression(max_iter=1000, class_weight="balanced")
bert_lr.fit(X_train_bert, y_train)

y_pred_bert = bert_lr.predict(X_test_bert)
print(classification_report(y_test, y_pred_bert))

              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        39
     Neutral       1.00      1.00      1.00        21
    Positive       1.00      1.00      1.00        40

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



MODEL COMPARISON
| Model | Features | Purpose |
|-----|---------|--------|
| Logistic Regression | TF-IDF | Baseline ML |
| BERT + Logistic | Transformer embeddings | Context-aware comparison |


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

nltk.download('stopwords')
nltk.download('wordnet')

st.set_page_config(page_title="AI Echo Dashboard", layout="wide")
st.title("🧠 AI Echo – Sentiment Analysis Dashboard")

# ---------------- Load Data ----------------
df = pd.read_excel("chatgpt_style_reviews_dataset.xlsx")

# ---------------- Sentiment Label ----------------
def rating_to_sentiment(r):
    if r >= 4:
        return "Positive"
    elif r == 3:
        return "Neutral"
    else:
        return "Negative"

df["sentiment"] = df["rating"].apply(rating_to_sentiment)

# ---------------- Text Cleaning ----------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["clean_review"] = df["review"].apply(clean_text)

# ---------------- Sidebar ----------------
option = st.sidebar.selectbox(
    "Select Question",
    [
        "1. Overall Sentiment",
        "2. Sentiment vs Rating",
        "3. Keyword Analysis",
        "4. Sentiment Over Time",
        "5. Verified Users",
        "6. Review Length",
        "7. Location Analysis",
        "8. Platform Analysis",
        "9. Version Analysis",
        "10. Negative Feedback Themes",
        "Live Prediction"
    ]
)

# 1️⃣ Overall Sentiment
if option == "1. Overall Sentiment":
    counts = df["sentiment"].value_counts()
    fig, ax = plt.subplots()
    ax.pie(counts, labels=counts.index, autopct="%1.1f%%")
    ax.set_title("Overall Sentiment Distribution")
    st.pyplot(fig)

# 2️⃣ Sentiment vs Rating
elif option == "2. Sentiment vs Rating":
    ct = pd.crosstab(df["rating"], df["sentiment"])
    st.dataframe(ct)
    st.bar_chart(ct)

# 3️⃣ Keyword Analysis
elif option == "3. Keyword Analysis":
    s = st.selectbox("Choose Sentiment", df["sentiment"].unique())
    text = " ".join(df[df.sentiment == s]["clean_review"])
    wc = WordCloud(width=800, height=400).generate(text)
    st.image(wc.to_array())

# 4️⃣ Sentiment Over Time
elif option == "4. Sentiment Over Time":
    df["date"] = pd.to_datetime(df["date"])
    df["month"] = df["date"].dt.to_period("M")
    trend = df.groupby(["month", "sentiment"]).size().unstack().fillna(0)
    st.line_chart(trend)

# 5️⃣ Verified Users
elif option == "5. Verified Users":
    ct = pd.crosstab(df["verified_purchase"], df["sentiment"])
    st.dataframe(ct)
    st.bar_chart(ct)

# 6️⃣ Review Length
elif option == "6. Review Length":
    df["review_length"] = df["review"].astype(str).apply(len)
    fig, ax = plt.subplots()
    sns.boxplot(data=df, x="sentiment", y="review_length", ax=ax)
    st.pyplot(fig)

# 7️⃣ Location Analysis
elif option == "7. Location Analysis":
    loc = df.groupby(["location", "sentiment"]).size().unstack().fillna(0)
    st.dataframe(loc.head(15))

# 8️⃣ Platform Analysis
elif option == "8. Platform Analysis":
    ct = pd.crosstab(df["platform"], df["sentiment"])
    st.bar_chart(ct)

# 9️⃣ Version Analysis
elif option == "9. Version Analysis":
    ct = pd.crosstab(df["version"], df["sentiment"])
    st.bar_chart(ct)

# 🔟 Negative Feedback Themes
elif option == "10. Negative Feedback Themes":
    neg_text = " ".join(df[df.sentiment == "Negative"]["clean_review"])
    wc = WordCloud(width=800, height=400).generate(neg_text)
    st.image(wc.to_array())

# 🔮 Live Prediction
elif option == "Live Prediction":
    X = df["clean_review"]
    y = df["sentiment"]

    tfidf = TfidfVectorizer(max_features=5000)
    X_vec = tfidf.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_vec, y)

    user_text = st.text_area("Enter review text")
    if st.button("Predict"):
        clean = clean_text(user_text)
        vec = tfidf.transform([clean])
        pred = model.predict(vec)[0]
        st.success(f"Predicted Sentiment: {pred}")


Overwriting app.py


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

In [ ]:
# 🧹 Step 1: Clean up old installations and config
!pip uninstall -y pyngrok
!rm -rf /root/.ngrok2
!pip install pyngrok --quiet

# 🧠 Step 2: Import and add your valid authtoken
from pyngrok import ngrok
!ngrok config add-authtoken "34yRMi4zjk8EouYfzRqonOioZtm_3sCnf2k5ZYEVTsKUA7uUk"

# 🚀 Step 3: Run Streamlit + open tunnel correctly
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(f"✅ Streamlit App Running at: {public_url.public_url}")

Found existing installation: pyngrok 7.5.0
Uninstalling pyngrok-7.5.0:
  Successfully uninstalled pyngrok-7.5.0
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Streamlit App Running at: https://subpopular-predominant-everette.ngrok-free.dev
